In [32]:
import numpy as np
from scipy.linalg import expm
import itertools as itt

## Gate definitions

We define the X pi/2 rotation on Q1 and Q2 and the Z phi rotation on Q1 and Q2. 

Our parameters are unitary errors away from target. They are stored in a dict with hashes that are the gate and qubit and values that are the parameters

In [33]:
SigX = np.array([[0, 1], [1, 0]])
SigY = np.array([[0, -1j], [1j, 0]])
SigZ = np.array([[1, 0], [0, -1]])
SigI = np.array([[1, 0], [0, 1]])

def model_Gxpi2(params, target_qubit=0, num_qubits=2):
    assert num_qubits == 2
    x0 = params[0]
    x1 = params[1]
    assert len(params) == 2
    Ux = expm(-(1j/2)*((np.pi/2 + x0)*SigX + x1*SigZ))
    if target_qubit == 0:
        return np.kron(Ux, SigI)
    elif target_qubit == 1:
        return np.kron(SigI, Ux)
    else:
        raise ValueError('target_qubit must be 0 or 1')


def model_Gzphi(phi, params, target_qubit=0, num_qubits=2):
    x0 = params[0]
    assert len(params) == 1
    Uz = expm(-(1j/2)*(phi+x0)*SigZ)
    if target_qubit == 0:
        return np.kron(Uz, SigI)
    elif target_qubit == 1:
        return np.kron(SigI, Uz)
    else:
        raise ValueError('target_qubit must be 0 or 1')
    
def model_Gzpi2(params, target_qubit=0, num_qubits=2):
    assert num_qubits == 2
    x0 = params[0]
    assert len(params) == 1
    Uz = expm(-(1j/2)*(np.pi/2+x0)*SigZ)
    if target_qubit == 0:
        return np.kron(Uz, SigI)
    elif target_qubit == 1:
        return np.kron(SigI, Uz)
    else:
        raise ValueError('target_qubit must be 0 or 1')

In [34]:
def sequence_to_unitary(sequence, param_dict):
    """
    Returns the unitary matrix corresponding to the sequence of gates in the sequence list

    Args:
        sequence: list of tuples, each tuple is the label of a gate and the target qubit
        param_dict: dictionary of parameters for each gate
    """
    num_qubits = 2
    U = np.eye(2**num_qubits)
    for gate, target in sequence:
        if gate == 'Xpi2':
            U = model_Gxpi2(param_dict[gate+f'_Q{target}'], target, num_qubits) @ U
        elif gate == 'Zphi':
            raise NotImplementedError('Gzphi not implemented')
        elif gate == 'Xpi2':
            U = model_Gzpi2(param_dict[gate+f'_Q{target}'], target, num_qubits) @ U
        else:
            raise ValueError('Invalid gate label')
    return U

In [35]:
def make_random_gateset_coherent_error_params(error_rate_1q, error_rate_2q):
    """
    Returns a dictionary of random parameters for a coherent error model

    Args:
        error_rate_1q: float, error rate for 1-qubit gates
        error_rate_2q: float, error rate for 2-qubit gates
    """
    param_dict = {}
    for gate in ['Xpi2']:
        for qubit in [0, 1]:
            param_dict[gate+f'_Q{qubit}'] = error_rate_1q*np.random.randn(2)
    for gate in ['Zpi2']:
        for qubit in [0, 1]:
            param_dict[gate+f'_Q{qubit}'] = error_rate_1q*np.random.randn(1)
    return param_dict

def make_target_params():
    """
    Returns a dictionary of target parameters for a coherent error model
    """
    param_dict = {}
    for gate in ['Xpi2']:
        for qubit in [0, 1]:
            param_dict[gate+f'_Q{qubit}'] = np.zeros(2)
    for gate in ['Zpi2']:
        for qubit in [0, 1]:
            param_dict[gate+f'_Q{qubit}'] = np.zeros(1)
    return param_dict

def check_unitary(U):
    """
    Returns True if the input matrix is unitary, False otherwise
    """
    return np.allclose(np.eye(U.shape[0]), U @ U.conj().T)

In [36]:
random_param_dict = make_random_gateset_coherent_error_params(0.001, 0.01)
TARGET_PARAMS = make_target_params()

In [37]:
sequence = [('Xpi2', 0), ('Xpi2', 1)]
params = {'Xpi2_Q0': [0, 0], 'Xpi2_Q1': [0, 0]}
U = sequence_to_unitary(sequence, random_param_dict)
print(check_unitary(U))
print(U.shape)

True
(4, 4)


In [ ]:
def calc_invariant_eigenphases

In [38]:
def calc_normalized_eigenmatrix(matrix):
    # Calculate the eigenvalues and eigenvectors of the matrix
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    # Normalize the eigenvectors
    eigen_norm = np.linalg.norm(eigenvectors, axis=1)
    # Return the normalized eigenvectors
    return eigenvectors/eigen_norm

def calc_eigen_jacobian_at_target(sequence, num_params, hilbert_dim=2):
    """
    Calculate the Jacobian of the eigenvalues of the target unitary matrix with respect to the parameters

    Returns a matrix of shape (hilbert_dim, num_params) where the (i, j) element is the derivative of the ith eigenvalue
    with respect to the jth parameter.
    """
    # Define the target unitary matrix
    target_params = np.zeros(num_params)
    target_unitary = sequence_to_unitary(sequence, TARGET_PARAMS)
    assert target_unitary.shape[0] == target_unitary.shape[1]
    assert target_unitary.shape[0] == hilbert_dim

    # Calculate the normalized eigenvectors of the target unitary matrix
    normalized_eigenvectors = calc_normalized_eigenmatrix(target_unitary)

    # Calculate the derivative of the eigenvalues with respect to the parameters
    jacobian = np.zeros((hilbert_dim, num_params), dtype=np.complex128)

    for i in range(hilbert_dim):
        # Calculate the derivs w.r.t. the ith parameter
        for j in range(num_params):
            # Define the parameter perturbations
            perturbations = np.zeros(num_params)
            perturbations[j] = 1e-8

            # Calculate the perturbed unitary matrices
            perturbed_unitary_plus = sequence_to_unitary(sequence, target_params + perturbations)
            perturbed_unitary_minus = sequence_to_unitary(sequence, target_params - perturbations)

            # Calculate the perturbed eigenvectors
            eval_plus = normalized_eigenvectors[i].conj().T @ perturbed_unitary_plus @ normalized_eigenvectors[i]
            eval_minus = normalized_eigenvectors[i].conj().T @ perturbed_unitary_minus @ normalized_eigenvectors[i]

            # Calculate the derivative
            jacobian[i, j] = (eval_plus - eval_minus)/(2*perturbations[j])
    return jacobian

In [39]:
random_param_dict

{'Xpi2_Q0': array([-4.34631294e-04,  4.60510771e-05]),
 'Xpi2_Q1': array([-0.00022603,  0.00257427]),
 'Zpi2_Q0': array([0.00049643]),
 'Zpi2_Q1': array([0.00111685])}

In [40]:
sequence = [('Xpi2', 0), ('Xpi2', 1)]
U = sequence_to_unitary(sequence, random_param_dict)
np.linalg.eig(U)

(array([3.29275776e-04-9.99999946e-01j, 3.29275776e-04+9.99999946e-01j,
        9.99999994e-01-1.05354836e-04j, 9.99999994e-01+1.05354836e-04j]),
 array([[ 0.50041693+0.00000000e+00j,  0.49958274+5.55111512e-17j,
         -0.50040226+2.21856130e-13j, -0.49959739-7.46236406e-13j],
        [ 0.49959739+6.82505268e-17j, -0.50040226+2.22044605e-16j,
         -0.49958274-1.02172502e-16j,  0.50041693+0.00000000e+00j],
        [ 0.50040226+5.78421859e-17j, -0.49959739+3.88578059e-16j,
          0.50041693+0.00000000e+00j, -0.49958274+1.27675648e-15j],
        [ 0.49958274-1.46313083e-16j,  0.50041693+0.00000000e+00j,
          0.49959739-2.22011333e-13j,  0.50040226+7.46402939e-13j]]))

In [41]:
calc_eigen_jacobian_at_target(sequence, 3, 2)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [20]:
debug

> /tmp/ipykernel_6441/1627711307.py(13)sequence_to_unitary()
     11     for gate, target in sequence:
     12         if gate == 'Xpi2':
---> 13             U = model_Gxpi2(param_dict[gate+f'_Q{target}'], target, num_qubits) @ U
     14         elif gate == 'Zphi':
     15             raise NotImplementedError('Gzphi not implemented')

'Xpi2'
'Xpi2'
array([0., 0., 0.])
array([0., 0., 0.])


In [10]:
calc_eigen_jacobian_at_target(['X', 'X', 'Z', 'Z'], 3, 2)

ValueError: not enough values to unpack (expected 2, got 1)

In [215]:
calc_eigen_jacobian_at_target(['Z', 'X', 'Z', 'Z', 'Z'], 3, 2)

ValueError: not enough values to unpack (expected 2, got 1)

In [216]:
calc_eigen_jacobian_at_target(['Z', 'Z', 'X', 'Z', 'Z', 'X'], 3, 2)

ValueError: not enough values to unpack (expected 2, got 1)

In [217]:
def score_eigen_jacobian_matrix(jacobian_matrix):
    svals = np.linalg.svd(jacobian_matrix, compute_uv=False)
    svals_clipped = np.clip(svals, 1e-8, None)
    return np.sum(np.log(svals_clipped))

In [218]:
sequences = [['X'], ['Z', 'X', 'Z', 'Z', 'Z'], ['Z', 'Z', 'X', 'Z', 'Z', 'X']]
num_params = 3
hilbert_dim = 2
total_jac = np.zeros((hilbert_dim, 0), dtype=np.complex128)
for sequence in sequences:
    jac = calc_eigen_jacobian_at_target(sequence, num_params, hilbert_dim)
    total_jac = np.concatenate((total_jac, jac), axis=1)
score_eigen_jacobian_matrix(total_jac)

ValueError: not enough values to unpack (expected 2, got 1)

In [219]:
score_eigen_jacobian_matrix(calc_eigen_jacobian_at_target(['X', 'X', 'Z', 'Z'], 3, 2))

ValueError: not enough values to unpack (expected 2, got 1)